#EDA Summary
	•	I had the U.S. and French election tables.
	•	I created flag variables to identify which articles came from which table and then merged them.
	•	The combined DataFrame included columns like: URL, date, time, language, description, relevance, and the flag variables.
	•	I looked at the language column first and saw that the majority of the articles were in English, with around 8% in French.
	•	I decided to filter and only keep English-language texts, which left me with 3,819 articles.
	•	I checked for duplicates and found two, so I dropped them and ended up with 3,817 articles.
	•	There were two types of relevance columns — one for relevance_description and one for relevance_text.
	•	I checked for null values and saw that relevance_description had around 2,000 nulls, while relevance_text only had 7 nulls.
	•	I decided to use the relevance_text column as the truth label for my classifier model, and I’ll generate the missing 7 with an LLM.
	•	Around 16.9% of the articles were marked as relevant, which is a small proportion — this means I’ll need to use some resampling techniques later.
	•	Then I looked at the URL column and extracted the publisher from it.
	•	Turns out we have 15 unique publishers, and the top three overall were Politico, Al Jazeera, and Euronews.
	•	I also checked which publishers published the most relevant articles — top three were NBC News, Washington Post, and NPR.
	•	I examined the text length of the articles. It followed a log distribution. Most of the articles had fewer than 2,000 words.
	•	I then looked at time trends to see when the peaks happened.
	•	There was a big spike around November 2024, which aligned with the U.S. elections.
	•	There was also a smaller spike around July 2024, which matched the French legislative elections.
	•	I checked the daily spikes, and they matched up with the actual election days, which is reassuring.
	•	Around 15% of the articles were flagged as relevant for the U.S. election, while less than 1% were flagged as relevant for the French election.
	•	That means it’s going to be harder to detect French election articles due to the smaller number of examples.


In [0]:
import pandas as pd
us_df = spark.table("int_aa_dev_common_worskapce_catalog.geopol.us_election_text").toPandas()
french_df = spark.table("int_aa_dev_common_worskapce_catalog.geopol.french_election_text").toPandas()

# Add one-hot style flags
us_df["is_us_election"] = 1
us_df["is_french_election"] = 0
us_df["is_german_election"] = 0

french_df["is_us_election"] = 0
french_df["is_french_election"] = 1
french_df["is_german_election"] = 0

# Combine them
df = pd.concat([us_df, french_df], ignore_index=True)

In [0]:
french_df.relevance_description.value_counts()

In [0]:
french_df.shape

In [0]:
df.head()

In [0]:
df.columns

language

In [0]:
df.language.value_counts()

In [0]:
# filter for eng lang 
df = df[df["language"] == "en"].copy()

count = df.count()
print(f"Total count: {count}")

duplicates

In [0]:
df.shape

In [0]:
df.duplicated().sum()

In [0]:
df[df.duplicated(keep=False)]

In [0]:
df = df.drop_duplicates(subset=["url"], keep="last")

In [0]:
df.shape

relevance description vs relevance text

In [0]:
df.relevance_description.value_counts(normalize = True)

In [0]:
df.relevance_description.isnull().sum()

In [0]:
df.relevance_text.value_counts(normalize = True)

In [0]:
generate_truth = df[df.relevance_text.isnull()]
generate_truth

url 

In [0]:
from urllib.parse import urlparse

# Example: assuming your DataFrame is called df and the URL column is named 'url'
def extract_domain(url):
    return urlparse(url).netloc

# Create a new column with just the domain names
df['publisher'] = df['url'].apply(extract_domain)

# Count how many unique news sources there are
unique_publishers = df['publisher'].nunique()
print(f"Number of unique publishers: {unique_publishers}")

# Optional: See the most common publishers
print(df['publisher'].value_counts())

relevance by publisher 

In [0]:
df.groupby("publisher")["relevance_text"].mean().sort_values(ascending=False)

Text lenght 

In [0]:
df["text_length"] = df["text"].str.split().str.len()
max_length = int(df["text_length"].max())
bins = range(0, max_length + 500, 500)    

df["text_length"].hist(bins=bins)
plt.xlabel("Text length (words)")
plt.ylabel("Number of articles")
plt.title("Distribution of Article Lengths")
plt.show()

time trends

In [0]:
df["date"] = pd.to_datetime(df["datetime"])
df.groupby(df["date"].dt.date)["relevance_text"].mean().plot()

In [0]:
daily_relevance = df.groupby(df["date"].dt.date)["relevance_text"].mean()
spike_days = daily_relevance.sort_values(ascending=False).head(10)
print(spike_days)

In [0]:
# First, filter to relevant articles only
relevant_df = df[df["relevance_text"] == 1]

# Then get value counts with ratios
relevant_ratio_by_flag = relevant_df[["is_us_election", "is_french_election"]].sum() / len(df)
print(relevant_ratio_by_flag)

# Feature engineering
	•	I used spaCy, an NLP library in Python, for text analysis.
	•	I used its en_core_web_sm pre-trained English language model.
	•	I wrote a function to generate reproducible features from the text data.
	•	The features included:
	•	News organization name, extracted from the article URL.
	•	Date-related features:
	•	Month
	•	Day of week
	•	Weekend indicator
	•	Election countdown flags:
	•	Number of days until the US and French elections.
	•	Word count of each article.
	•	Verb count, using spaCy’s part-of-speech tagging.
	•	Election keyword flag: if the article text mentions keywords like “election”, “vote”, “poll”, etc.
	•	Article type classification (e.g., analysis, interview, or other) based on keyword presence.
	•	Some features were left out for now because they depend on extracted entities, which I’m still generating. These include:
	•	Entity count
	•	Entity-to-word ratio
	•	Whether the article mentions any organizations
	•	After generating these features, I ran the function on my full DataFrame.
	•	I then saved the resulting featurized DataFrame to CSV for future modeling.


In [0]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

In [0]:
import pandas as pd
import numpy as np
import re
import spacy
from urllib.parse import urlparse
from datetime import datetime

# Load spaCy model (ensure it's installed with: python -m spacy download en_core_web_sm)
nlp = spacy.load("en_core_web_sm")

def generate_article_features(
    df,
    text_col="text",
    url_col="url",
    date_col="datetime",
    #entities_col="extracted_entities"
):
    df = df.copy()
    
    # Make sure date is datetime
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    # --- News org name (from URL)
    df["news_org"] = df[url_col].apply(lambda x: urlparse(x).netloc if pd.notnull(x) else None)

    # --- Date features
    df["date"] = df[date_col].dt.date
    df["day_of_week"] = df[date_col].dt.day_name()
    df["month"] = df[date_col].dt.month
    df["is_weekend"] = df[date_col].dt.weekday >= 5
    #df["time_published"] = df[date_col].dt.time

    # --- Election countdown flags (you can customize dates)
    us_election_date = pd.to_datetime("2024-11-05")
    french_election_date = pd.to_datetime("2024-07-07")
    df["days_until_us_election"] = (us_election_date - df[date_col]).dt.days
    df["days_until_french_election"] = (french_election_date - df[date_col]).dt.days

    # --- Word count
    df["word_count"] = df[text_col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

    # --- Verb count
    def count_verbs(text):
        if not isinstance(text, str): return 0
        doc = nlp(text)
        return sum(1 for token in doc if token.pos_ == "VERB")
    df["verb_count"] = df[text_col].apply(count_verbs)

    # # --- Entity count & ratio of entities to word count
    # def count_entities(entities):
    #     if not isinstance(entities, dict): return 0
    #     return sum(len(v) for v in entities.values())
    
    # df["entity_count"] = df[entities_col].apply(count_entities)
    # df["entity_to_word_ratio"] = df["entity_count"] / df["word_count"].replace(0, np.nan)

    # --- Election keyword flag
    election_keywords = ["election", "vote", "poll", "ballot", "campaign", "runoff"]
    df["mentions_election_keywords"] = df[text_col].str.contains("|".join(election_keywords), case=False, na=False)

    # # --- Mentions organization flag
    # df["mentions_organizations"] = df[entities_col].apply(
    #     lambda x: isinstance(x, dict) and len(x.get("institutions", [])) > 0
    # )

    # --- Language (pass-through if already exists, else dummy 'en')
    if "language" not in df.columns:
        df["language"] = "en"

    # --- Article type (simple heuristic)
    def get_article_type(text):
        if not isinstance(text, str): return "other"
        text = text.lower()
        if "analysis" in text: return "analysis"
        if "interview" in text: return "interview"
        return "other"
    df["article_type"] = df[text_col].apply(get_article_type)

    return df

In [0]:
df_featurized = generate_article_features(df)

In [0]:
df_featurized

In [0]:
df_featurized.to_csv("df_featurized_us_french_no_entities.csv", index=False)